In [ ]:
# Cell 1: Importera alla bibliotek vi behöver för u3 (SVM)

import pandas as pd              # Datahantering (DataFrames)
import numpy as np               # Numeriska beräkningar

from sklearn.model_selection import train_test_split    # Dela upp data i train/test
from sklearn.svm import SVC                             # Support Vector Classifier (SVM)
from sklearn.preprocessing import MinMaxScaler          # Normalisering till [0,1]
from sklearn.metrics import confusion_matrix, accuracy_score  # Mått på prestanda


In [ ]:
# Cell 2: Läs in datasetet student-mat och skapa features (X) och target (y)

# Läs in student-mat.csv (matematik-elever)
df = pd.read_csv("student-mat.csv", sep=";")

# Välj alla numeriska kolumner i datan
numeric_cols = df.select_dtypes(include=[np.number]).columns

# Features X = alla numeriska kolumner UTOM G3 (slutbetyget)
X = df[numeric_cols].drop(columns=["G3"])

# Target y = slutbetyg G3 omvandlat till tre klasser:
# 0–9   -> "low"
# 10–14 -> "medium"
# 15–20 -> "high"
bins = [-1, 9, 14, 20]                # Gränser för intervallen
labels = ["low", "medium", "high"]    # Klassnamn
y = pd.cut(df["G3"], bins=bins, labels=labels)

# Kolla att vi har rimligt med observationer i varje klass
print("Fördelning av klasser i y:")
print(y.value_counts())
print("\nForm på X (antal rader, antal features):", X.shape)


In [ ]:
# Cell 3: Funktion som gör EN körning med SVM (SVC)

def run_svm(X, y, test_size, normalize=False, kernel="linear", random_state=42):
    """
    Gör EN körning med SVM (sklearn.svm.SVC).

    Parametrar:
    - X : features (predictors)
    - y : target (klasserna low/medium/high)
    - test_size : andel av datan som ska vara test (t.ex. 0.25 eller 0.30)
    - normalize : True => normalisera X till [0,1] (MinMaxScaler)
    - kernel : vilken kernel-funktion vi använder, t.ex. 'linear' eller 'rbf'
    - random_state : används för train_test_split för reproducerbarhet

    Returnerar:
    - cm  : confusion matrix (3x3, ordning: low, medium, high)
    - acc : accuracy på testdatan
    """

    # 1. Dela upp data i train och test
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=test_size,
        stratify=y,          # Behåll samma klassfördelning i train och test
        random_state=random_state
    )

    # 2. Normalisera features om vi valt det
    if normalize:
        scaler = MinMaxScaler(feature_range=(0, 1))
        X_train = scaler.fit_transform(X_train)   # Lär sig min/max från träningsdatan
        X_test = scaler.transform(X_test)         # Använder samma skalning på testdatan
    else:
        X_train = X_train.values
        X_test = X_test.values

    # 3. Skapa och träna SVM-modellen
    # C=1.0, gamma='scale' är standard; vi varierar bara kernel enligt uppgift
    svm_clf = SVC(kernel=kernel)
    svm_clf.fit(X_train, y_train)

    # 4. Prediktion på testdatan
    y_pred = svm_clf.predict(X_test)

    # 5. Confusion matrix och accuracy
    cm = confusion_matrix(y_test, y_pred, labels=y.cat.categories)
    acc = accuracy_score(y_test, y_pred)

    return cm, acc


In [ ]:
# Cell 4: Kör alla SVM-exekveringar och skriv ut confusion matrix + accuracy

# Två olika train/test-splittar
splits = [
    (0.25, "train=75%, test=25%"),
    (0.30, "train=70%, test=30%")
]

# Två datatyper: originaldata och normaliserat [0,1]
data_settings = [
    (False, "originaldata"),
    (True,  "normaliserat [0,1]")
]

# Två olika kernel-funktioner
kernels = ["linear", "rbf"]

# Lista där vi sparar alla resultat (för tabell + top 3)
results = []

for test_size, split_text in splits:
    for normalize_flag, data_text in data_settings:
        for kernel in kernels:
            cm, acc = run_svm(
                X, y,
                test_size=test_size,
                normalize=normalize_flag,
                kernel=kernel,
                random_state=42
            )

            print("--------------------------------------------------")
            print(f"Kernel = {kernel}, data = {data_text}, split = {split_text}")
            print("Confusion matrix (rader/kolumner: low, medium, high):")
            print(cm)
            print(f"Accuracy: {acc:.4f}")

            results.append({
                "kernel": kernel,
                "data_typ": data_text,
                "split_text": split_text,
                "test_size": test_size,
                "normalize": normalize_flag,
                "accuracy": acc,
                "confusion_matrix": cm
            })


In [ ]:
# Cell 5: Skapa tabell med alla körningar och ta fram de tre bästa

# Gör DataFrame utan själva confusion_matrix-objekten
res_df = pd.DataFrame([
    {k: v for k, v in r.items() if k != "confusion_matrix"}
    for r in results
])

print("Alla SVM-körningar:")
display(res_df[["kernel", "data_typ", "split_text", "accuracy"]])

# Sortera efter accuracy (högst först) och ta de tre bästa körningarna
top3 = res_df.sort_values(by="accuracy", ascending=False).head(3)

print("\nTre bästa körningar (enligt accuracy):")
display(top3)


In [ ]:
# Cell 6: Visa confusion matrices för de tre bästa körningarna

print("Confusion matrices för de tre bästa SVM-körningarna:\n")

for i, row in top3.reset_index(drop=True).iterrows():
    best_cm = None
    for r in results:
        if (
            r["kernel"] == row["kernel"] and
            r["data_typ"] == row["data_typ"] and
            r["split_text"] == row["split_text"] and
            abs(r["accuracy"] - row["accuracy"]) < 1e-9
        ):
            best_cm = r["confusion_matrix"]
            break

    print("==============================================")
    print(f"Körning {i+1}:")
    print(f"Kernel = {row['kernel']}, data = {row['data_typ']}, split = {row['split_text']}")
    print(f"Accuracy: {row['accuracy']:.4f}")
    print("Confusion matrix (low, medium, high):")
    print(best_cm)
